In [2]:
import requests as rq
import bs4 as bs
import json
import traceback
import re
import numpy as np
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import glob, os

## 1. BBC

In [4]:
links_to_visit = {}

In [5]:
def get_link(url):
    home_bbc = rq.get(url)
    soup = bs.BeautifulSoup(home_bbc.text, 'html.parser')
    for a in soup.find_all('a', href=True):
        if a.text:
            link = a["href"]
            if link.startswith("/swahili/habari-") or link.startswith("/swahili/michezo-"):
                if link not in list(links_to_visit.keys()):
                    links_to_visit[link] = False

In [6]:
get_link("https://www.bbc.com/swahili")
get_link("https://www.bbc.com/swahili/michezo")
len(links_to_visit)

47

In [7]:
for link in list(links_to_visit.keys()):
    link_url = "https://www.bbc.com/" + link
    if links_to_visit[link] == False:
        get_link(link_url)
        links_to_visit[link] = True
        
        no_link = all(value == True for value in list(links_to_visit.values()))
        if no_link:
            break

In [8]:
len(list(links_to_visit))

104

## 2. RFI

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [2]:
links_visit_rfi = {}
load_more = "btn-load-more alt more-loader"

seed = ["http://sw.rfi.fr/ulaya/all/", 
        "http://sw.rfi.fr/amerika/all/", 
        "http://sw.rfi.fr/mashariki-ya-kati/all/",
        "http://sw.rfi.fr/asia/all/",
        "http://sw.rfi.fr/afrika/all/",
        "http://sw.rfi.fr/eac/all/",
        "http://sw.rfi.fr/siasa-uchumi/all/",
        "http://sw.rfi.fr/afya-mazingira/all/",
        "http://sw.rfi.fr/utamaduni/all/",
        "http://sw.rfi.fr/michezo/all/"]

driver = webdriver.Chrome(executable_path="/home/aims/Downloads/chromedriver")
driver.get("http://sw.rfi.fr/afrika/all/")
loadmore = driver.find_element_by_class_name("cookie-bar_hide")
loadmore.click()
while True:
    try:
        time.sleep(1)
        loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "btn-load-more")))
        loadmore.click()
        time.sleep(1)
    except Exception:
        print("Reached bottom of page")
        traceback.print_exc()
        break

soup_rfi = bs.BeautifulSoup(driver.page_source,'html.parser')
# home_rfi.text
# soup_rfi = bs.BeautifulSoup(home_rfi.text, 'html.parser')
news_links = soup_rfi.find_all('a', href=True)
len(news_links)

Reached bottom of page


Traceback (most recent call last):
  File "<ipython-input-2-e97f09742b74>", line 22, in <module>
    loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "btn-load-more")))
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/support/wait.py", line 71, in until
    value = method(self._driver)
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/support/expected_conditions.py", line 128, in __call__
    return _element_if_visible(_find_element(driver, self.locator))
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/support/expected_conditions.py", line 415, in _find_element
    raise e
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/support/expected_conditions.py", line 411, in _find_element
    return driver.find_element(*by)
  File "/home/aims/.local/lib/python3.7/site-packages/selenium/webdriver/remote/webdriver.py", line 978, in find_element
    'value': v

WebDriverException: Message: chrome not reachable
  (Session info: chrome=79.0.3945.88)


In [31]:
def save_data(data, filename="final.txt"):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s" % item)

In [74]:
news_final = []

for a in news_links:
    if a.text:
        link = a["href"]
        matching = re.match(r'^\/michezo\/\d+\w+', link)
        if matching:
            new_link = 'http://sw.rfi.fr'+link
            news_final.append(new_link)

save_data(news_final, 'michezo.txt')

In [97]:
os.chdir("./")
final_data = []
for file in glob.glob("*.txt"):
    with open(file) as f:
        for line in f:
            if line not in final_data:
                final_data.append(line)

In [98]:
save_data(final_data, 'news_final.txt')
len(final_data)

19593

In [3]:
urls = []
def get_urls():
    with open("news_final.txt") as f:
        for line in f:
            urls.append(line)

In [4]:
get_urls()

In [5]:
import asyncio
import concurrent.futures
import requests

final_data = []

def do_req(url):
    return requests.get(url, headers=headers)

async def main():

    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                do_req,
                url
            )
            for url in urls
        ]
        for response in await asyncio.gather(*futures):
            page_soup = bs.BeautifulSoup(response.text, 'html.parser')
            news_title = page_soup.select_one('article h1')
            news_title = news_title.text if news_title else ''
            news_body = page_soup.select_one('article div[itemprop="articleBody"]')
            news_body = news_body.text if news_body else ''
            final_data.append({'title' : news_title, 'content' : news_body})


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [10]:
full = []
for item in final_data:
    if item['content'] != '':
        full.append(item)

In [11]:
with open('data_full.json', 'w') as fout:
    json.dump(full, fout)

## 3. VOA

### 3.1 Swahili

In [37]:
home_voa_swahili = rq.get("https://www.voaswahili.com/")

In [38]:
home_voa_swahili

<Response [200]>

### 3.2 Lingala

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [32]:
def save_data(data, filename="final.txt"):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)

In [ ]:
links_visit_rfi = {}
load_more = "btn-load-more alt more-loader"

seed = ["http://sw.rfi.fr/ulaya/all/", 
        "http://sw.rfi.fr/amerika/all/", 
        "http://sw.rfi.fr/mashariki-ya-kati/all/",
        "http://sw.rfi.fr/asia/all/",
        "http://sw.rfi.fr/afrika/all/",
        "http://sw.rfi.fr/eac/all/",
        "http://sw.rfi.fr/siasa-uchumi/all/",
        "http://sw.rfi.fr/afya-mazingira/all/",
        "http://sw.rfi.fr/utamaduni/all/",
        "http://sw.rfi.fr/michezo/all/"]

for i in range(len(seed)):
    driver = webdriver.Chrome(executable_path="/home/aims/Downloads/chromedriver")
    driver.get("http://sw.rfi.fr/afrika/all/")
    loadmore = driver.find_element_by_class_name("cookie-bar_hide")
    loadmore.click()
    while True:
        try:
            time.sleep(1)
            loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "btn-load-more")))
            loadmore.click()
            time.sleep(1)
        except Exception:
            print("Reached bottom of page")
            traceback.print_exc()
            break

    soup_rfi = bs.BeautifulSoup(driver.page_source,'html.parser')
    # home_rfi.text
    # soup_rfi = bs.BeautifulSoup(home_rfi.text, 'html.parser')
    news_links = soup_rfi.find_all('a', href=True)
    len(news_links)

In [59]:
# home_voa_lingala = rq.get("https://www.voalingala.com") / https://www.voalingala.com/z/5744
links_visit_rfi = {}
load_more = "btn-load-more alt more-loader"

seed = ["http:///www.voalingala.com/z/5746",
        "http:///www.voalingala.com/z/5745",
        "http:///www.voalingala.com/z/5744",
        "http:///www.voalingala.com/z/5776",
        "http:///www.voalingala.com/z/5747",
        "http:///www.voalingala.com/z/5748",
        "http:///www.voalingala.com/z/5749",
        "http:///www.voalingala.com/z/5760",
        "http:///www.voalingala.com/z/5756",
        "http:///www.voalingala.com/z/5753",
        "http:///www.voalingala.com/z/5761",
        "http:///www.voalingala.com/z/5759",
        "http:///www.voalingala.com/z/5751",
        "http:///www.voalingala.com/z/5757",
        "http:///www.voalingala.com/z/5755",
        "http:///www.voalingala.com/z/5758",
        "http:///www.voalingala.com/z/5754",
        "http:///www.voalingala.com/z/5752"]

driver = webdriver.Chrome(executable_path="./ChromeDriver/chromedriver_linux64/chromedriver")
driver.get(seed[0])
while True:
    try:
        time.sleep(1)
        loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "link-showMore")))
        loadmore.click()
        time.sleep(1)
    except Exception:
        print("Reached bottom of page")
        traceback.print_exc()
        break

Reached bottom of page


Traceback (most recent call last):
  File "<ipython-input-59-0c64fe89a9cd>", line 29, in <module>
    loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "link-showMore")))
  File "/home/aims/anaconda3/envs/aims/lib/python3.7/site-packages/selenium/webdriver/support/wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



In [62]:
# home_voa_lingala = rq.get("https://www.voalingala.com") / https://www.voalingala.com/z/5744
links_visit_rfi = {}
load_more = "btn-load-more alt more-loader"

seed = ["http:///www.voalingala.com/z/5746",
        "http:///www.voalingala.com/z/5745",
        "http:///www.voalingala.com/z/5744",
        "http:///www.voalingala.com/z/5776",
        "http:///www.voalingala.com/z/5747",
        "http:///www.voalingala.com/z/5748",
        "http:///www.voalingala.com/z/5749",
        "http:///www.voalingala.com/z/5760",
        "http:///www.voalingala.com/z/5756",
        "http:///www.voalingala.com/z/5753",
        "http:///www.voalingala.com/z/5761",
        "http:///www.voalingala.com/z/5759",
        "http:///www.voalingala.com/z/5751",
        "http:///www.voalingala.com/z/5757",
        "http:///www.voalingala.com/z/5755",
        "http:///www.voalingala.com/z/5758",
        "http:///www.voalingala.com/z/5754",
        "http:///www.voalingala.com/z/5752"]

for i in range(len(seed)):
    links_visit_rfi = {}
    load_more = "btn-load-more alt more-loader"
    
    driver = webdriver.Chrome(executable_path="./ChromeDriver/chromedriver_linux64/chromedriver")
    driver.get(seed[i])
    while True:
        try:
            time.sleep(1)
            loadmore = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CLASS_NAME, "link-showMore")))
            loadmore.click()
            time.sleep(1)
        except Exception:
            print("Reached bottom of page")
            break

    soup_rfi = bs.BeautifulSoup(driver.page_source,'html.parser')
    # home_rfi.text
    # soup_rfi = bs.BeautifulSoup(home_rfi.text, 'html.parser')
    news_links = soup_rfi.find_all('a', href=True)
    print(f' We got {len(news_links)} links')
    
    news_final = []

    for a in news_links:
        if a.text:
            link = a["href"]
            matching = re.match(r'^\/a\/\w+', link) # r'^\/z\/\d+\w+, r'^\/z\/\d+$
            if matching:
                new_link = 'http://www.voalingala.com'+link
                if new_link not in news_final:
                    news_final.append(new_link)
    name = 'categorie_'+str(i)+'.txt'
    save_data(news_final, name)

Reached bottom of page
 We got 158 links
Reached bottom of page
 We got 294 links
Reached bottom of page
 We got 2100 links
Reached bottom of page
 We got 162 links
Reached bottom of page
 We got 360 links
Reached bottom of page
 We got 236 links
Reached bottom of page
 We got 150 links
Reached bottom of page
 We got 98 links
Reached bottom of page
 We got 200 links
Reached bottom of page
 We got 300 links
Reached bottom of page
 We got 90 links
Reached bottom of page
 We got 438 links
Reached bottom of page
 We got 1322 links
Reached bottom of page
 We got 134 links
Reached bottom of page
 We got 1954 links
Reached bottom of page
 We got 948 links
Reached bottom of page
 We got 114 links
Reached bottom of page
 We got 228 links


In [88]:
glob.glob("*.txt")

['categorie_15.txt',
 'categorie_9.txt',
 'categorie_12.txt',
 'categorie_3.txt',
 'categorie_1.txt',
 'categorie_8.txt',
 'categorie_11.txt',
 'categorie_7.txt',
 'categorie_16.txt',
 'categorie_6.txt',
 'categorie_13.txt',
 'categorie_17.txt',
 'categorie_14.txt',
 'categorie_10.txt',
 'categorie_0.txt',
 'categorie_5.txt',
 'categorie_4.txt',
 'categorie_2.txt']

In [85]:
os.chdir("./Lingala_links/")
final_data = []
for file in glob.glob("*.txt"):
    with open(file) as f:
        for line in f:
            if line not in final_data:
                final_data.append(line)

In [90]:
save_data(final_data, 'news_all_lingala.txt')
len(final_data)

1972

In [125]:
# os.chdir("../") # Back to the main repository, dont run twice
urls = []
def get_urls():
    with open("./news_all_lingala.txt") as f:
        for line in f :
            if line != '\n':
                urls.append(line[:-1]) # Remove the /n at the queue of the string :) 

In [126]:
get_urls()

In [127]:
urls[:5]

['http://www.voalingala.com/a/jolino-makelele-alimbola-mokano-ya-mbulamatari-mpo-na-unikin/5236898.html',
 'http://www.voalingala.com/a/tshisekedi-aboyi-unikin-ekangama/5235985.html',
 'http://www.voalingala.com/a/bato-11-bakufi-na-boloko-ya-makala-mpo-na-bozangi-nkisi/5235621.html',
 'http://www.voalingala.com/a/moko-na-ba-avocats-ya-babanyamulenge-ayanoli-mpo-etali-milice-gumino/5235345.html',
 'http://www.voalingala.com/a/bapoliciers-babale-bakufi-na-mobulu-ya-ba-%C3%A9tudiants-na-unikin-(pnc)/5235327.html']

In [173]:
import asyncio
import concurrent.futures
import requests

final_data = []

def do_req(url):
    return requests.get(url, headers=headers)

async def main():

    with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                do_req,
                url
            )
            for url in urls
        ]
        for response in await asyncio.gather(*futures):
            soup = bs.BeautifulSoup(response.text, 'html.parser')
            # get title
            title = soup.select_one('div.col-title h1')
            description = soup.select_one('div.media-pholder--video  div.intro')
            if not description:
                description = soup.select_one('div#article-content .wsw')
                
            # news_body = news_body.text if news_body else ''
            final_data.append({'title' : title, 'description' : description})


loop = asyncio.get_event_loop()
loop.run_until_complete(main())

RuntimeError: This event loop is already running

In [162]:
urls[3]

'http://www.voalingala.com/a/moko-na-ba-avocats-ya-babanyamulenge-ayanoli-mpo-etali-milice-gumino/5235345.html'

In [16]:
# for i in range(20):
import requests as rq
import bs4 as bs
home_bbc = rq.get('https://www.voalingala.com/a/sassou-andimami-candidat-ya-pct-mpo-na-pr%C3%A9sidentielle-ya-2021/5226931.html')
soup = bs.BeautifulSoup(home_bbc.text, 'html.parser')

# get title
title = soup.select_one('div.col-title h1')
description = soup.select_one('div.media-pholder--video  div.intro')
if not description:
    description = soup.select_one('div#article-content .wsw')
    

# get description
        
# <div class="intro m-t-md">
# <p>Ba étudiants bazali kobima na ba homes ya Campus ya Université ya Kinshasa uto butu boleki kolanda mokano ya mbulamatari nsima na mobulu ya mokolo mwa yambo mpe ya mibale. VOA Lingala ebengaki molebeli ya mbulamatari mpo kolimbola mokano yango. Toyoka ministre Jolilono Makelele.</p>
# </div>

In [17]:
title.text, description.text

('\nSassou andimami candidat ya PCT mpo na présidentielle ya 2021\n',
 '\n"Bato 2.588 bazalaki na Congrès bandimi bango banso ete camarade Denis Sassou N’Guesso azonga na bokambi ya comité central ya lingomba mpe azala candidat na biso na maponami ya mokonzi ya mboka na 2021," elobi Pierre Ngolo, kalaka mokonzi ya PCT oyo atiki ebonga na ye na Pierre Moussa.\nM. Sassou azali na mibu tuku sambo na mitano, azali kokamba Congo-Brazzaville uto mibu tuku misato na mitano.\n"Tokanisi ete tokoki te kotika Denis Sassou N’Guesso. Esengelakii kaka kopona ye. Ezali mpo na mbongwana kati na maye mazali kokobai, ezali mpo na kimya mpe boboto na mboka," eyebisi na AFP Léonidas Mottom, questeur molandi na Assemblée nationale mpo na PCT.\nBuku likonzi epesi M. Sassou nzela ya kopesa lisusu nkombo na ye na maponami ya 2021 mpe mpo na mbala ya suka, na 2026.\nUPADS (Union panafricaine pour la démocratie sociale), lingomba monene ya opposition, esengaki jeudi ete maponami mapusana na 2023 mpe M. Sassou a

In [9]:
with open("data_lig.json", "r") as read_file:
    data = json.load(read_file)

FileNotFoundError: [Errno 2] No such file or directory: 'data_lig.json'

In [12]:
with open('sw_final.txt', 'w') as file:
    for item in data:
        join_title_description = item['title'] + '\n\n' + item['content'] + '\n\n'
        file.write(join_title_description)